In [70]:
!pip install yfinance
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.6
    Uninstalling Werkzeug-2.3.6:
      Successfully uninstalled Werkzeug-2.3.6


In [61]:
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sbn
import plotly.graph_objects as go
from datetime import datetime

import numpy as np
import pandas as pd

In [3]:
df = yf.download('GOOG', start='2018-01-01', end='2023-07-30')

[*********************100%***********************]  1 of 1 completed


In [4]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1402.000000,1402.000000,1402.000000,1402.000000,1402.000000,1.402000e+03
mean,88.191864,89.223590,87.254528,88.254749,88.254749,3.075255e+07
std,30.656664,30.978960,30.328614,30.641826,30.641826,1.334256e+07
min,48.695000,50.176998,48.505501,48.811001,48.811001,6.936000e+06
25%,59.175001,59.742376,58.592625,59.251624,59.251624,2.225828e+07
50%,79.675747,80.822502,78.926750,79.608749,79.608749,2.738145e+07
75%,114.444998,116.320251,113.395449,114.822876,114.822876,3.479412e+07
max,151.863495,152.100006,149.887497,150.709000,150.709000,1.241400e+08


In [5]:
df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [6]:
df.dtypes

Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1402 entries, 2018-01-02 to 2023-07-28
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1402 non-null   float64
 1   High       1402 non-null   float64
 2   Low        1402 non-null   float64
 3   Close      1402 non-null   float64
 4   Adj Close  1402 non-null   float64
 5   Volume     1402 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 76.7 KB


In [8]:
# Utilizar plotly express para crear el gráfico de líneas
fig = px.line(df, y=['Adj Close', 'Open'], title='Gráfico de líneas para Adj Close y Open')

# Mostrar el gráfico
fig.show()

In [46]:
df_retorno = df.copy()
df_retorno['Retorno'] = np.log(df_retorno['Adj Close']).diff().cumsum()
df_retorno = df_retorno.dropna()
df_retorno

,Open,High,Low,Close,Adj Close,Volume,Retorno
Date,,,,,,,
2018-01-03,53.215500,54.314499,53.160500,54.124001,54.124001,28604000,0.016280
2018-01-04,54.400002,54.678501,54.200100,54.320000,54.320000,20092000,0.019895
2018-01-05,54.700001,55.212502,54.599998,55.111500,55.111500,25582000,0.034361
2018-01-08,55.111500,55.563499,55.081001,55.347000,55.347000,20952000,0.038625
2018-01-09,55.470001,55.528500,55.061550,55.313000,55.313000,18050000,0.038010
...,...,...,...,...,...,...,...
2023-07-24,121.926003,123.349998,121.379997,121.879997,121.879997,22276100,0.828039
2023-07-25,121.879997,123.690002,121.529999,122.790001,122.790001,31820800,0.835478
2023-07-26,130.360001,131.369995,128.710007,129.660004,129.660004,46216900,0.889918


In [47]:
# Crear la gráfica de rendimiento total
empresa = "Google"
fig = px.line(data_frame=df, x=df_retorno.index, y=df_retorno['Retorno'], title=f"Rendimiento Total de {empresa}")
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="Rendimiento Total")
fig.show()

In [48]:
# Calcular el rendimiento total con los datos de Yahoo Finance
rendimiento_total_yahoo = df["Adj Close"].pct_change().cumsum()
rendimiento_logaritmico = np.log(df["Adj Close"]).diff().cumsum()


In [56]:
# Asegurarse de que rendimiento_total_yahoo tenga la misma longitud que df_retorno
rendimiento_total_yahoo = rendimiento_total_yahoo[:len(df_retorno)]

# Crear la gráfica de comparación
fig = px.line(data_frame=df_retorno, x=df_retorno.index, y=['Retorno'],
              labels={"variable": "Fuente de Datos", "value": "Rendimiento Total"},
              title=f"Comparación del Rendimiento Total de {empresa}")

# Agregar los datos de rendimiento_total_yahoo a la gráfica
fig.add_scatter(x=df_retorno.index, y=rendimiento_total_yahoo, mode='lines', name='rendimiento_total_yahoo')

fig.update_xaxes(title_text="Fecha")
fig.show()


In [69]:
#Grafica de velas japonesas
fig = go.Figure(data=[go.Candlestick(x=df_retorno.index,
                open=df_retorno['Open'], high=df_retorno['High'],
                low=df_retorno['Low'], close=df_retorno['Adj Close'],
                text=df_retorno['Retorno'].apply(lambda x: f"Retorno: {x:.2f}"))])

fig.update_layout(title="Gráfico de Velas Japonesas con Retornos",
                  xaxis_title="Fecha",
                  yaxis_title="Precios")

fig.show()



In [20]:
# Calcular la volatilidad
volatilidad = df_retorno["Retorno"].std() * (252 ** 0.5)  # 252 días hábiles en un año

# Crear una serie con la misma longitud y el mismo índice que el DataFrame df
volatilidad_series = pd.Series(volatilidad, index=df.index)

In [23]:
fig = px.line(data_frame=df, x=df.index, y=volatilidad_series, title=f"Volatilidad de {empresa}")
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="Volatilidad")
fig.show()

In [24]:
# Eliminar filas con valores faltantes en la columna "Rendimiento Total"
#df.dropna(subset=["Rendimiento Total"], inplace=True)
df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [28]:
# Calcular la tasa de crecimiento anual
valor_inicial = df_retorno["Retorno"].iloc[1]
valor_final = df_retorno["Retorno"].iloc[-1]
tasa_crecimiento_anual = (valor_final / valor_inicial) ** (1 / 6) - 1

# Crear una serie con la misma longitud y el mismo índice que el DataFrame df
crecimiento_series = pd.Series(tasa_crecimiento_anual, index=df.index)

In [29]:
fig = px.line(data_frame=df, x=df.index, y=crecimiento_series, title=f"Tasa de Crecimiento de {empresa}")
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="Crecimiento")
fig.show()